# Data Science Unit 1 Sprint Challenge 4

## Exploring Data, Testing Hypotheses

In this sprint challenge you will look at a dataset of people being approved or rejected for credit.

https://archive.ics.uci.edu/ml/datasets/Credit+Approval

Data Set Information: This file concerns credit card applications. All attribute names and values have been changed to meaningless symbols to protect confidentiality of the data. This dataset is interesting because there is a good mix of attributes -- continuous, nominal with small numbers of values, and nominal with larger numbers of values. There are also a few missing values.

Attribute Information:
- A1: b, a.
- A2: continuous.
- A3: continuous.
- A4: u, y, l, t.
- A5: g, p, gg.
- A6: c, d, cc, i, j, k, m, r, q, w, x, e, aa, ff.
- A7: v, h, bb, j, n, z, dd, ff, o.
- A8: continuous.
- A9: t, f.
- A10: t, f.
- A11: continuous.
- A12: t, f.
- A13: g, p, s.
- A14: continuous.
- A15: continuous.
- A16: +,- (class attribute)

Yes, most of that doesn't mean anything. A16 (the class attribute) is the most interesting, as it separates the 307 approved cases from the 383 rejected cases. The remaining variables have been obfuscated for privacy - a challenge you may have to deal with in your data science career.

Sprint challenges are evaluated based on satisfactory completion of each part. It is suggested you work through it in order, getting each aspect reasonably working, before trying to deeply explore, iterate, or refine any given step. Once you get to the end, if you want to go back and improve things, go for it!

## Part 1 - Load and validate the data

- Load the data as a `pandas` data frame.
- Validate that it has the appropriate number of observations (you can check the raw file, and also read the dataset description from UCI).
- UCI says there should be missing data - check, and if necessary change the data so pandas recognizes it as na
- Make sure that the loaded features are of the types described above (continuous values should be treated as float), and correct as necessary

This is review, but skills that you'll use at the start of any data exploration. Further, you may have to do some investigation to figure out which file to load from - that is part of the puzzle.

In [1]:
import pandas as pd
import numpy as np

In [2]:
original = pd.read_csv(
    'https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data',
    names = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 
             'A11', 'A12', 'A13', 'A14', 'A15', 'A16'],
    na_values = '?',
    dtype = {'A2': np.float64, 'A3': np.float64, 'A8': np.float64, 
             'A11': np.int32, 'A14': np.float64, 'A15': np.int32})
original.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,280.0,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,+


In [3]:
original.describe()

,A2,A3,A8,A11,A14,A15
count,678.000000,690.000000,690.000000,690.00000,677.000000,690.000000
mean,31.568171,4.758725,2.223406,2.40000,184.014771,1017.385507
std,11.957862,4.978163,3.346513,4.86294,173.806768,5210.102598
min,13.750000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,22.602500,1.000000,0.165000,0.00000,75.000000,0.000000
50%,28.460000,2.750000,1.000000,0.00000,160.000000,5.000000
75%,38.230000,7.207500,2.625000,3.00000,276.000000,395.500000
max,80.250000,28.000000,28.500000,67.00000,2000.000000,100000.000000


In [4]:
original.isnull().sum()

A1     12
A2     12
A3      0
A4      6
A5      6
A6      9
A7      9
A8      0
A9      0
A10     0
A11     0
A12     0
A13     0
A14    13
A15     0
A16     0
dtype: int64

In [5]:
# UCI says the shape of the data is:
#  Number of Instances: 690
#  Number of Attributes: 15 + class attribute
# And it matches the shape we see.
original.shape

(690, 16)

In [6]:
# Since there are very few nulls, compared to
# the number of rows (690), we will drop them.
original.dropna(inplace = True)

In [7]:
original.isnull().sum().sum()

0

## Part 2 - Exploring data, Testing hypotheses

The only thing we really know about this data is that A16 is the class label. Besides that, we have 6 continuous (float) features and 9 categorical features.

Explore the data: you can use whatever approach (tables, utility functions, visualizations) to get an impression of the distributions and relationships of the variables. In general, your goal is to understand how the features are different when grouped by the two class labels (`+` and `-`).

For the 6 continuous features, how are they different when split between the two class labels? Choose two features to run t-tests (again split by class label) - specifically, select one feature that is *extremely* different between the classes, and another feature that is notably less different (though perhaps still "statistically significantly" different). You may have to explore more than two features to do this.

For the categorical features, explore by creating "cross tabs" between them and the class label, and apply the Chi-squared test to them. There are 9 categorical features - as with the t-test, try to find one where the Chi-squared test returns an extreme result (rejecting the null that the data are independent), and one where it is less extreme.

**NOTE** - "less extreme" just means smaller test statistic/larger p-value. Even the least extreme differences may be strongly statistically significant.

Your *main* goal is the hypothesis tests, so don't spend too much time on the exploration/visualization piece. That is just a means to an end. This is challenging, so manage your time and aim for a baseline of at least running two t-tests and two Chi-squared tests before polishing. And don't forget to answer the questions in part 3, even if your results in this part aren't what you want them to be.

In [8]:
from scipy import stats

In [9]:
numeric_cols = ['A2', 'A3', 'A8', 'A11', 'A14', 'A15']
categorical_cols = ['A1', 'A4', 'A5', 'A6', 'A7', 'A9', 'A10', 
                    'A12', 'A13']

def get_col_from_group(grouped_obj, group_name, col):
    for name, group in grouped_obj:
        if (name == group_name):
            return group[col]
    raise(ValueError, 'Could not find column in group')

grouped = original.groupby('A16')
for col in numeric_cols:
    credit_approved = get_col_from_group(grouped, '+', col)
    credit_rejected = get_col_from_group(grouped, '-', col)
    cs, pvalue = stats.ttest_ind(credit_approved, credit_rejected)
    print('col:', col, 'cs:', cs, 'pvalue:', pvalue)
    
# From the result below:
# A11 feature is notably different, and the difference is statistically significant
# A2 feature is not notably different, but the difference is statistically significant

col: A2 cs: 4.675662433767456 pvalue: 3.5636710933835225e-06
col: A3 cs: 5.400813416576192 pvalue: 9.310154396147606e-08
col: A8 cs: 9.002392498622463 pvalue: 2.4079238505859142e-18
col: A11 cs: 11.336964562239809 pvalue: 2.5864649620278843e-27
col: A14 cs: -2.18221936438708 pvalue: 0.029450100409286473
col: A15 cs: 4.475369764700449 pvalue: 9.003915641872878e-06


In [10]:
df_categorical = original[categorical_cols]
pd.crosstab([original['A1'], original['A4'], 
              original['A5'], original['A6'], 
              original['A7'], original['A9'], 
              original['A10'], original['A12'], 
              original['A13']],
            original['A16'])
# As you can see, crosstabs are difficult to look at as a whole.
# Let's try to get the answer programmatically.
# Problem running chisquare test (see below).
# Select feature and analyze using crosstab.

A16                            +  -
A1 A4 A5 A6 A7 A9 A10 A12 A13      
a  l  gg cc ff f  f   t   s    1  0
   u  g  aa h  f  f   f   g    0  1
               t  f   f   g    0  1
            v  f  f   f   g    0  4
                      t   g    0  1
                  t   t   g    0  2
               t  f   f   g    3  0
                      t   g    1  0
                  t   f   g    4  0
                      t   g    2  0
         c  bb t  t   t   g    1  0
            h  t  f   f   g    1  0
                      t   g    1  1
                  t   f   g    2  0
            v  f  f   f   g    0  6
                      t   g    0  1
                  t   t   g    0  3
               t  f   f   g    1  0
                  t   f   g    4  0
                      t   g    4  1
         cc h  t  t   f   g    2  0
                      t   g    3  0
            v  f  f   f   g    1  0
               t  f   f   g    2  0
                      t   g    0  1
         d  v  f  f   f   g    0  2
                      t   g    0  2
         e  bb t  t   f   g    1  0
            dd t  t   t   g    1  0
            v  t  f   t   g    1  0
...                           .. ..
b  y  p  i  h  t  t   t   g    0  1
            v  f  f   t   g    0  1
         j  v  f  f   f   g    0  1
         k  ff f  f   f   g    0  1
            h  f  f   t   g    0  1
               t  f   f   g    1  0
                      t   g    0  1
            v  f  f   f   g    0  5
                          s    0  1
                  t   t   g    0  1
               t  t   f   g    1  1
                      t   g    1  0
         m  bb f  f   t   s    1  0
               t  t   t   g    1  0
            v  f  f   f   g    0  2
                          s    0  1
                      t   g    0  2
                          s    0  1
         q  h  f  t   t   g    0  1
         r  n  f  f   t   g    0  1
         w  h  t  f   t   g    1  0
            v  f  f   f   g    0  1
                          s    0  2
                      t   g    0  2
               t  f   t   g    3  0
                  t   f   g    2  0
                      t   g    2  1
         x  h  f  f   f   g    0  1
               t  f   t   g    0  1
            v  t  t   t   g    2  0

[348 rows x 2 columns]

In [11]:
pd.crosstab([original['A1'], original['A4']], original['A16'])
# Looks like a possibly good feature is A1 == a and A4 == y (around 60% credit disapproved)
# Another similar one is A1 == b and A4 == y

A16      +    -
A1 A4          
a  l     1    0
   u    83   80
   y    11   28
b  l     1    0
   u   166  170
   y    34   79

In [12]:
pd.crosstab([original['A5'], original['A6']], original['A16'])
# Good credit features:
#   A5 == g and A6 == cc (almost 75% if favor of good credit)
# Bad credit features:
#   A5 == g and A6 == ff
#   A5 == p and A6 == ff (or k)

A16     +   -
A5 A6        
g  aa  16  23
   c   49  49
   cc  25   6
   d    4  16
   e   12   7
   ff   6  26
   i   13  33
   j    2   3
   k   10  22
   m   14  16
   q   46  23
   r    2   0
   w   25  24
   x   25   2
gg cc   1   0
   ff   1   0
p  aa   3  10
   c   11  24
   cc   3   5
   d    3   3
   e    2   3
   ff   0  17
   i    1   8
   j    1   4
   k    3  13
   m    2   6
   q    3   3
   r    0   1
   w    8   6
   x    5   4

In [13]:
pd.crosstab([original['A7'], original['A9']], original['A16'])
# Looks like a good split with the A7 == 'bb' and A9 == f.  Most of these are negative credit.
# A7 == ff and A9 == f is also good with most negative credit.
# So is At == h and A9 == f
# Also A7 == v and A9 == f

A16      +    -
A7 A9          
bb f     2   19
   t    22   10
dd f     0    4
   t     2    0
ff f     1   40
   t     7    6
h  f     2   37
   t    85   13
j  f     1    4
   t     2    1
n  f     1    2
   t     1    0
o  f     1    1
v  f    10  178
   t   153   40
z  f     0    1
   t     6    1

In [14]:
pd.crosstab([original['A10'], original['A12']], original['A16'])
# Bad credit features:
# A10 == f
# Good credit features:
# A10 == t

A16        +    -
A10 A12          
f   f     44  156
    t     49  117
t   f    107   44
    t     96   40

In [15]:
pd.crosstab(original['A13'], original['A16'])

A16,+,-
A13,,
g,280,318
p,1,1
s,15,38


In [16]:
for col in categorical_cols:
    credit_approved = get_col_from_group(grouped, '+', col)
    credit_rejected = get_col_from_group(grouped, '-', col)
    a = credit_approved.values
    b = credit_rejected.values
    if a.shape != b.shape:
        print('shapes of + and - sections of col:', col, 'are unequal - cannot do chisquare test.')
        continue
        
    res = stats.chisquare(np.array([a, b]))
    print(res)

# From the result below, we can see that since the shapes of the
# two sections of each feature are different, we cannot do the
# chisquare test on them.  Back to crosstab interpretation.

shapes of + and - sections of col: A1 are unequal - cannot do chisquare test.
shapes of + and - sections of col: A4 are unequal - cannot do chisquare test.
shapes of + and - sections of col: A5 are unequal - cannot do chisquare test.
shapes of + and - sections of col: A6 are unequal - cannot do chisquare test.
shapes of + and - sections of col: A7 are unequal - cannot do chisquare test.
shapes of + and - sections of col: A9 are unequal - cannot do chisquare test.
shapes of + and - sections of col: A10 are unequal - cannot do chisquare test.
shapes of + and - sections of col: A12 are unequal - cannot do chisquare test.
shapes of + and - sections of col: A13 are unequal - cannot do chisquare test.


## Part 3 - Analysis and Interpretation

Now that you've looked at the data, answer the following questions:

- Interpret and explain the two t-tests you ran - what do they tell you about the relationships between the continuous features you selected and the class labels?
- Interpret and explain the two Chi-squared tests you ran - what do they tell you about the relationships between the categorical features you selected and the class labels?
- What was the most challenging part of this sprint challenge?

Answer with text, but feel free to intersperse example code/results or refer to it from earlier.

### T-test interpretation

The A11 feature is notably different, and the difference is statistically significant.  This is one feature I would pick to find meaning in this dataset.


A2 feature is not notably different, but the difference is statistically significant.  This is not a feature I would pick to find meaning in this dataset.

### Chi-squared tests

Could not run them because the split between the + and - groups did not have the same number of values.  The chisquare function could not broadcast to the right shape, and so it failed.  This was the most challenging part - trying to figure out how to get the chisquare test to work (unfortunately, still don't know).

### Cross-tab interpretation


Good credit features:
  A5 == g and A6 == cc (almost 75% if favor of good credit)
Bad credit features:
  A5 == g and A6 == ff
  A5 == p and A6 == ff (or k)
  
Looks like a good split with the A7 == 'bb' and A9 == f.  Most of these are negative credit.
A7 == ff and A9 == f is also good with most negative credit.
So is At == h and A9 == f
Also A7 == v and A9 == f

Bad credit features:
  A10 == f
Good credit features:
  A10 == t